<a href="https://colab.research.google.com/github/marclamberts/football-analysis/blob/main/Box_Score_%2B_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

# Load the match event CSV
file_path = '/content/drive/MyDrive/Recruitment/Brøndby 1-1 Nordsjælland.csv'
df = pd.read_csv(file_path)

In [ ]:
import pandas as pd

# Load your CSV file (update the file path as needed)
df = pd.read_csv("Brøndby 1-1 Nordsjælland.csv")

# Select relevant columns for the box score
box_score_columns = [
    'playerId', 'playerName', 'typeId', 'outcome', 'keyPass', 'assist'
]

# Filter rows with valid player data
player_events = df[box_score_columns].dropna(subset=['playerId', 'playerName'])

# Define basic box score metrics
player_events['is_pass'] = player_events['typeId'] == 1
player_events['successful_pass'] = (player_events['typeId'] == 1) & (player_events['outcome'] == 1)
player_events['key_pass'] = player_events['keyPass'] == 1
player_events['assist'] = player_events['assist'] == 1
player_events['shots'] = player_events['typeId'] == 13
player_events['successful_take_on'] = (player_events['typeId'] == 3) & (player_events['outcome'] == 1)
player_events['failed_take_on'] = (player_events['typeId'] == 3) & (player_events['outcome'] == 0)
player_events['tackle'] = player_events['typeId'] == 4
player_events['foul'] = player_events['typeId'] == 7

# Group by player to summarize actions
box_score_summary = player_events.groupby(['playerId', 'playerName']).agg({
    'is_pass': 'sum',
    'successful_pass': 'sum',
    'key_pass': 'sum',
    'assist': 'sum',
    'shots': 'sum',
    'successful_take_on': 'sum',
    'failed_take_on': 'sum',
    'tackle': 'sum',
    'foul': 'sum'
}).reset_index()

# Rename columns for clarity
box_score_summary.columns = [
    'Player ID', 'Player Name', 'Passes Attempted', 'Passes Completed',
    'Key Passes', 'Assists', 'Shots', 'Successful Take-Ons',
    'Failed Take-Ons', 'Tackles', 'Fouls Committed'
]

# Define weights for BPM components
weights = {
    'Passes Completed': 0.05,
    'Key Passes': 0.3,
    'Assists': 0.6,
    'Shots': 0.4,
    'Successful Take-Ons': 0.3,
    'Failed Take-Ons': -0.2,
    'Tackles': 0.4,
    'Fouls Committed': -0.5
}

# Calculate Football BPM using a weighted sum of box score components
box_score_summary['Football BPM'] = (
    box_score_summary['Passes Completed'] * weights['Passes Completed'] +
    box_score_summary['Key Passes'] * weights['Key Passes'] +
    box_score_summary['Assists'] * weights['Assists'] +
    box_score_summary['Shots'] * weights['Shots'] +
    box_score_summary['Successful Take-Ons'] * weights['Successful Take-Ons'] +
    box_score_summary['Failed Take-Ons'] * weights['Failed Take-Ons'] +
    box_score_summary['Tackles'] * weights['Tackles'] +
    box_score_summary['Fouls Committed'] * weights['Fouls Committed']
)

# Sort players by Football BPM
box_score_bpm = box_score_summary.sort_values(by='Football BPM', ascending=False)

# Show final output
print(box_score_bpm[['Player Name', 'Football BPM']])
